In [1]:
import os
import glob
import pickle
import methpype
import pandas as pd
import numpy as np

In [2]:
data_dir = "/data/Genetic/tab_data"

In [12]:
cellc = pd.read_csv("/data/Genetic/tab_data/EstimateCellCounts_PPMI_EPICn524final030618.csv")
pat_stat = pd.read_csv("/data/Genetic/tab_data/Patient_Status.csv")
pat_info = pd.read_csv("/data/Genetic/tab_data/PPMI_Meth_n524_for_LONI030718.tsv", delimiter='\t')
raw_dat = pd.read_csv("/data/Genetic/tab_data/beta_post_Funnorm_PPMI_EPICn524final030618.csv")

In [4]:
pat_numbers = list(pat_info["PATNO"].astype(int))
is_PD = lambda id: pat_stat[pat_stat.PATNO == id]["RECRUITMENT_CAT"].values[0] == 'PD'
get_sentrix = lambda id: str(pat_info[pat_info.PATNO == id]["Sentrix ID"].values[0]) + "_" + pat_info[pat_info.PATNO == id]["Sentrix Position"].values[0]
getXVal_cellc = lambda id: cellc[cellc.Sentrix_position == get_sentrix(id)].values[0][1:].astype(float)
getXVal_raw = lambda id: raw_dat[get_sentrix(id)].values

In [5]:
X = np.zeros((len(pat_numbers), 864067))
y = np.zeros((len(pat_numbers)), dtype=int)
for i, number in enumerate(pat_numbers):
    if(is_PD(number)):
        y[i] = 1
    X[i] = getXVal_raw(number)

7

In [6]:
from sklearn.decomposition import PCA as pca
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split as splt
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import matthews_corrcoef as mcc

forest = rfc(n_estimators = 1000, n_jobs = -1)
decomp = pca(n_components=32, svd_solver='full')
X_tr, X_ts, y_tr, y_ts = splt(X, y, test_size = 0.2, random_state = 42, stratify = y)
decomp.fit(X_tr)
X_tr_decomp = decomp.transform(X_tr)
X_ts_decomp = decomp.transform(X_ts)
forest.fit(X_tr_decomp, y_tr)
y_ts_pred = forest.predict(X_ts_decomp)

In [7]:
forest.feature_importances_

array([0.03286611, 0.03674732, 0.03217759, 0.04026861, 0.04795287,
       0.03806641, 0.03469936, 0.0392332 , 0.02687767, 0.03407445,
       0.03615581, 0.03253822, 0.03341689, 0.02943701, 0.02917566,
       0.0267704 , 0.02884632, 0.02816539, 0.03261977, 0.02906725,
       0.02457295, 0.02646324, 0.02477451, 0.03166111, 0.04365383,
       0.02724019, 0.02618613, 0.02456822, 0.02391346, 0.02579352,
       0.02279893, 0.02921762])

In [8]:
temp_arr = decomp.explained_variance_ * forest.feature_importances_

In [9]:
decomp.components_.shape

(32, 864067)

In [10]:
feature_importances = np.matmul(temp_arr.reshape((1,32)), decomp.components_)[0]

In [13]:
raw_dat.index.values.shape

(864067,)

In [14]:
feature_importances.shape

(864067,)

In [15]:
feature_dic = dict(zip(raw_dat.index.values, abs(feature_importances)))

In [16]:
feature_dic

{'cg14817997': 0.01642728885633913,
 'cg26928153': 0.0063535129258867895,
 'cg16269199': 0.02150481171868129,
 'cg13869341': 0.0021468782639308963,
 'cg14008030': 0.011576357512808054,
 'cg12045430': 0.009415331532703526,
 'cg20826792': 0.0008714110282282742,
 'cg20253340': 0.015039572264735776,
 'cg02404219': 0.04781509502432363,
 'cg21870274': 0.016802154518095753,
 'cg04098293': 0.014162938869389078,
 'cg16382250': 0.011270060911660239,
 'cg24335620': 0.0017137299973847132,
 'cg22463138': 0.005650957381740784,
 'cg16162899': 0.028858592479198676,
 'cg01803908': 0.001162961479735311,
 'cg17149495': 0.03160655879254363,
 'cg22802167': 0.0024741887997790308,
 'cg24669183': 0.0036057149263096257,
 'cg17308840': 0.005454657864142817,
 'cg17866181': 0.017286995088305675,
 'cg25018832': 0.000659663206779698,
 'cg26679879': 0.019371162877362534,
 'cg22519184': 0.013406148382724315,
 'cg11894324': 0.0005009776108737665,
 'cg24159721': 0.0018052182485688842,
 'cg02288058': 0.01629523719862576

In [17]:
import operator
sorted_features_value = sorted(feature_dic.items(), key = lambda item: -1 * item[1])

In [18]:
sorted_features = [feature[0] for feature in sorted_features_value]

In [19]:
sorted_features_value

[('cg15925199', 0.0975659934134949),
 ('cg25205946', 0.09750248802824743),
 ('cg10252249', 0.09649752687731579),
 ('cg22028367', 0.0941556063763069),
 ('cg17544920', 0.09342244825040238),
 ('cg08417382', 0.09326192922418887),
 ('cg16767700', 0.09243670802926608),
 ('cg07851521', 0.09196413574490842),
 ('cg27277239', 0.09174736413472422),
 ('cg04820593', 0.09133660032052576),
 ('cg23680829', 0.09118205192875022),
 ('cg17797028', 0.09020469240993928),
 ('cg06907892', 0.08974384507015454),
 ('cg26439324', 0.08961797345453278),
 ('cg16764023', 0.08961018424227929),
 ('cg14804181', 0.0894437181206964),
 ('cg23762217', 0.08875856184025765),
 ('cg15774752', 0.08815927111998167),
 ('cg03670113', 0.08813296856693417),
 ('cg03685263', 0.08712989536247326),
 ('cg27488875', 0.08711070669968321),
 ('cg07799180', 0.08680208956145075),
 ('cg01807421', 0.08657967776743726),
 ('cg05257947', 0.08646839109089569),
 ('cg24446128', 0.08644757418346061),
 ('cg10717149', 0.08641403497006953),
 ('cg07795413',

In [21]:
pickle.dump(sorted_features_value, open('/data/Genetic/compiled_info/genetic_importances.pkl', 'wb'))